In [ ]:
from molass import get_version
assert get_version() >= '0.7.4', "this notebook requires molass v0.7.4 or higher."
from molass_data import SAMPLE4
from molass.DataObjects import SecSaxsData as SSD
ssd = SSD(SAMPLE4)
trimmed_ssd = ssd.trimmed_copy()
corrected_ssd = trimmed_ssd.corrected_copy()
rgcurve = corrected_ssd.xr.compute_rgcurve()
decomposition = corrected_ssd.quick_decomposition(proportions=[0.2, 0.5, 0.3])
decomposition.plot_components(rgcurve=rgcurve)

In [ ]:
decomposition.get_rgs()

In [ ]:
from importlib import reload
import molass.Rigorous.RigorousImplement as RigorousImplement
reload(RigorousImplement)
from molass.Rigorous.RigorousImplement import make_basecurves_from_decomposition
baseline_objects, baseparams = make_basecurves_from_decomposition(decomposition, debug=True)

In [ ]:
baseparams

In [ ]:
init_params = decomposition.make_rigorous_initparams(baseparams, debug=False)
init_params

In [ ]:
from importlib import reload
import molass.Rigorous.RigorousImplement as RigorousImplement
reload(RigorousImplement)
from molass.Rigorous.RigorousImplement import make_dsets_from_decomposition
dsets = make_dsets_from_decomposition(decomposition, rgcurve, debug=False)
print(f"Number of datasets: D.shape={dsets[0][1].shape}, U.shape={dsets[2][1].shape}")


In [ ]:
import molass.Rigorous.RigorousImplement as RigorousImplement
reload(RigorousImplement)
from molass.Rigorous.RigorousImplement import construct_legacy_optimizer
spectral_vectors = decomposition.ssd.get_spectral_vectors()
model = decomposition.xr_ccurves[0].model
optimizer = construct_legacy_optimizer(dsets, baseline_objects, spectral_vectors, num_components=decomposition.num_components, model=model, method='BH', debug=False)

In [ ]:
from molass_legacy.Optimizer.JobStatePlot import plot_objective_func
optimizer.prepare_for_optimization(init_params)
plot_objective_func(optimizer, init_params)